In [ ]:
# This notebook is an experiement that implements LlamaIndex's use case on augment image analysis
# that constructes a ReAct agent (Chain of Thoughts)
# https://docs.llamaindex.ai/en/stable/examples/multi_modal/mm_agent.html
# because of the unaccessibility of the EXAtool API key, this notebook has not been finished
# Could be a fun use case for RAG playground

In [ ]:
# Web Agent: Given text/images, can query a web tool to lookup relevant information from the web (given a picture of shoes).

In [2]:
# get api_key from .env
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
# openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
!pip3 install llama-hub
!pip3 install exa_py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 MB 20.3 MB/s eta 0:00:0000:0100:01
  Using cached llama_index-0.9.40-py3-none-any.whl (15.9 MB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl (25 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 KB 40.1 MB/s eta 0:00:00
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 KB 25.3 MB/s eta 0:00:00
  Using cached dataclasses_json-0.6.3-py3-none-any.whl (28 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.2/269.2 KB 24.5 MB/s eta 0:00:00
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Using cached mars

  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8
    Uninstalling networkx-2.8:
      Successfully uninstalled networkx-2.8
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.5
    Uninstalling nest-asyncio-1.5.5:
      Successfully uninstalled nest-asyncio-1.5.5
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.1
    Uninstalling aiohttp-3.8.1:
      Successfully uninstalled aiohttp-3.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.3.0 requires pydantic!=1.8,!=1

In [13]:
from llama_hub.tools.exa import ExaToolSpec
from llama_index.agent.react_multimodal.step import MultimodalReActAgentWorker
from llama_index.agent import AgentRunner
from llama_index.multi_modal_llms import MultiModalLLM, OpenAIMultiModal
from llama_index.agent import Task

In [14]:
# https://llamahub.ai/l/tools-exa?from=all
# This tool connects to Exa to easily enable your agent to search and get HTML content from the Internet.
exa_tool_spec = ExaToolSpec(
    api_key=os.environ['OPENAI_API_KEY'],
)
exa_tools = exa_tool_spec.to_tool_list()

In [15]:
mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", max_new_tokens=1000)

In [16]:
# Initialize AgentRunner with OpenAIAgentWorker
react_step_engine = MultimodalReActAgentWorker.from_tools(
    exa_tools,
    # [],
    multi_modal_llm=mm_llm,
    verbose=True,
)
agent = AgentRunner(react_step_engine)

In [23]:
from llama_index.schema import ImageDocument

query_str = "Look up some reviews regarding these shoes."
image_document = ImageDocument(image_path="./addidas.png")

task = agent.create_task(
    query_str, extra_state={"image_docs": [image_document]}
)

In [24]:
def execute_step(agent: AgentRunner, task: Task):
    step_output = agent.run_step(task.task_id)
    if step_output.is_last:
        response = agent.finalize_response(task.task_id)
        print(f"> Agent finished: {str(response)}")
        return response
    else:
        return None


def execute_steps(agent: AgentRunner, task: Task):
    response = execute_step(agent, task)
    while response is None:
        response = execute_step(agent, task)
    return response

In [25]:
# no EXA api key here
response = execute_step(agent, task)

Thought: The image shows a pair of white sneakers with black stripes, which appear to be Adidas Superstars. I need to use a tool to search for reviews of these shoes.
Action: search
Action Input: {'query': 'Adidas Superstar sneakers reviews'}


ValueError: Request failed with status code 401: {"error":"Invalid API key. API key is not a UUID"}

In [26]:
response = execute_step(agent, task)

IndexError: pop from an empty deque